<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [23]:
# Install spark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [24]:
import findspark
findspark.init()

In [25]:
# Start session

In [ ]:
# Now create a new SparkContext
sc = SparkContext()
# Creating a spark session
spark = SparkSession.builder.appName("SparkML Ops Sample").getOrCreate()

In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2024-10-09 08:48:44--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K   265KB/s    in 0.9s    

2024-10-09 08:48:48 (265 KB/s) - ‘searchterms.csv’ saved [233457/233457]



In [ ]:
# Load the csv into a spark dataframe

In [28]:
df = spark.read.csv("searchTerms.csv", header=True, inferSchema=True)

In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [30]:
# Print the number of rows and columns
print(f"Number of rows: {df.count()}, Number of columns: {len(df.columns)}")  # {{ edit_1 }}

Number of rows: 10000, Number of columns: 4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [31]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [34]:
# Find out the datatype of the column searchterm
df.printSchema()  # {{ edit_2 }}

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- searchterm: string (nullable = true)



In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [37]:
# Count how many times the term 'gaming laptop' was searched
df.filter(df['searchterm'] == 'gaming laptop').count()

499

In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [43]:
df.groupBy('searchterm').count().orderBy('count', ascending=False).limit(5).show()

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz


In [45]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz && tar -xzvf model.tar.gz  # {{ edit_2 }}

--2024-10-09 09:06:34--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2024-10-09 09:06:36 (474 MB/s) - ‘model.tar.gz’ saved [1490/1490]

x sales_prediction.model/
x sales_prediction.model/metadata/
x sales_prediction.model/metadata/part-00000
x sales_prediction.model/metadata/.part-00000.crc
x sales_prediction.model/metadata/_SUCCESS
x sales_prediction.model/data/
x sales_prediction.model/data/part-000

In [53]:
import tarfile

# Unzip the tar.gz file
with tarfile.open('model.tar.gz', 'r:gz') as tar: 
    tar.extractall(path='extracted_model')  

In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [54]:
from pyspark.ml.regression import LinearRegressionModel

model = LinearRegressionModel.load('extracted_model/sales_prediction.model')

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [61]:
from pyspark.ml.feature import VectorAssembler

def predict(year):
    # Create a VectorAssembler to convert the 'year' column into 'features'
    assembler = VectorAssembler(inputCols=["year"], outputCol="features")
    data = [[year]]
    columns = ["year"]
    df = spark.createDataFrame(data, columns)
    # Transform the DataFrame to include the 'features' column
    df_with_features = assembler.transform(df).select('features')
    # Use the model to predict
    predictions = model.transform(df_with_features)
    # Show the prediction
    predictions.select('prediction').show()

# Test
predict(2023)


+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



24/10/09 12:43:29 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 459780 ms exceeds timeout 120000 ms
24/10/09 12:43:29 WARN SparkContext: Killing executors is not supported by current scheduler.
24/10/09 12:43:37 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$